In [31]:
import sqlalchemy
from sqlalchemy import text
import psycopg2
from psycopg2.extensions import cursor
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import json
from typing import Generator
import requests
import re
import pandas as pd
import logging
import csv
import time
import json
from pybaseball import statcast_single_game, statcast_pitcher, playerid_reverse_lookup, playerid_lookup

In [23]:
database_username = 'postgres'
database_password = quote_plus("postgres")
database_ip       = '172.19.0.2'
database_name     = 'mlb_dw'
database_connection = sqlalchemy.create_engine('postgresql+psycopg2://{0}:{1}@{2}/{3}?options=-csearch_path%3Ddbo,raw'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_timeout = 28800, max_overflow = -1)
connection = database_connection.connect()
print("made connection")
connection.close()

made connection


In [6]:
with open('./data/sched_2014.json', 'r') as infile:
    sched_2014 = infile.read()
    sched_2014_json = json.loads(sched_2014)

In [24]:
def extract_games(schedule: json) -> Generator[dict, None, None]:
    for date in schedule['dates']:
        for game in date['games']:
            yield {'date':date['date'], 'gamePk':game['gamePk']}

In [8]:
ext_games_2014 = extract_games(sched_2014_json) 

In [9]:
print(next(ext_games_2014))

{'date': '2014-03-22', 'gamePk': 380537}


In [46]:
def load_games(games, database_connection, logfile, pk_list):
    for date in games:
        if str(date['gamePk']) not in pk_list:
            connection = database_connection.connect()
            #print("made connection")
            try:
                data = statcast_single_game(date['gamePk'])
                data.to_sql('game_data_2023', connection, if_exists='append')
                with open(f'./data/{logfile}', 'a') as outfile:
                    outfile.write(str(date['gamePk']))
            except Exception as e:
                print(f"error={e}, gamePk={date['gamePk']}")
                connection.close()
            connection.close()

In [11]:
game_396303 = statcast_single_game(396303)

In [12]:
game_396303

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
236,FF,2014-03-22,90.7,0.97,5.73,"Adduci, James",451192,525768,strikeout,swinging_strike,...,8,4,8,4,8,NaN,NaN,NaN,0.003,NaN
242,FF,2014-03-22,92.5,0.81,5.71,"Adduci, James",451192,525768,NaN,ball,...,8,4,8,4,8,NaN,NaN,NaN,0.000,NaN
250,FF,2014-03-22,91.4,0.89,5.77,"Adduci, James",451192,525768,NaN,foul,...,8,4,8,4,8,NaN,NaN,NaN,0.000,NaN
257,CU,2014-03-22,73.0,1.24,5.67,"Adduci, James",451192,525768,NaN,blocked_ball,...,8,4,8,4,8,NaN,NaN,NaN,0.000,NaN
264,CU,2014-03-22,72.7,1.37,5.67,"Adduci, James",451192,525768,NaN,ball,...,8,4,8,4,8,NaN,NaN,NaN,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,FF,2014-03-22,87.8,1.81,6.25,"Adduci, James",451192,450306,NaN,called_strike,...,0,0,0,0,0,NaN,NaN,NaN,0.000,NaN
248,FF,2014-03-22,86.4,2.00,6.31,"Andrus, Elvis",462101,450306,single,hit_into_play,...,0,0,0,0,0,NaN,NaN,NaN,-0.025,NaN
255,FF,2014-03-22,85.3,1.95,6.41,"Profar, Jurickson",595777,450306,field_out,hit_into_play,...,0,0,0,0,0,NaN,NaN,NaN,0.022,NaN
269,FC,2014-03-22,85.6,1.90,6.31,"Profar, Jurickson",595777,450306,NaN,ball,...,0,0,0,0,0,NaN,NaN,NaN,0.000,NaN


In [16]:
connection = database_connection.connect()
game_396303.to_sql('game_data', connection, if_exists='append')
connection.close()

In [26]:
r = requests.get(url='https://statsapi.mlb.com/api/v1/schedule?sportId=1&startDate=2023-03-30&endDate=2023-07-06')

In [27]:
sched_2023_07_06_json = r.json()

In [28]:
ext_games_2023_07_06 = extract_games(sched_2023_07_06_json)

In [47]:
load_games(ext_games_2023_07_06, database_connection, 'inserted_2023_07_06.txt', pk_list)

error='NoneType' object has no attribute 'to_sql', gamePk=718700
error='NoneType' object has no attribute 'to_sql', gamePk=718558
error='NoneType' object has no attribute 'to_sql', gamePk=718371
error='NoneType' object has no attribute 'to_sql', gamePk=717664


In [38]:
with open(f'./data/inserted_2023_07_06.txt', 'r') as infile:
    loaded = infile.read()

In [39]:
pk_list = []

In [40]:
8004 / 6

1334.0

In [41]:
for i in range(0,1334):
    pk_list.append(loaded[:6])
    loaded = loaded[6:]

In [43]:
len(pk_list)

1334

In [44]:
pk_list[-1]

'717462'

In [45]:
next_game = next(ext_games_2023_07_06)
print(next_game['gamePk'])
print(str(next_game['gamePk']) in pk_list)

718773
True


In [50]:
playerid_lookup('Montero', 'Miguel')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,montero,miguel,471083,montm001,montemi01,3364,2006.0,2018.0


In [28]:
resp = requests.get(url='https://statsapi.mlb.com/api/v1/game/381165/boxscore')
boxscores_381165 = resp.json()
players_game_381165 = boxscores_381165['teams']['away']['players']
players_list = []
for k,v in players_game_381165.items():
    players_list.append(v)

In [3]:
boxscores_381165 = resp.json()

In [10]:
players_game_381165 = boxscores_381165['teams']['away']['players']

In [11]:
players_list = []
for k,v in players_game_381165.items():
    players_list.append(v)

In [12]:
players_list[0]['person']

{'id': 456776, 'fullName': 'Alex Torres', 'link': '/api/v1/people/456776'}

In [17]:
conn = database_connection.connect()
cur = conn.cursor()
cur.close()
conn.close()

AttributeError: 'Connection' object has no attribute 'cursor'

In [20]:
database_username = 'postgres'
database_password = 'postgres'
database_ip       = '172.19.0.2'
database_name     = 'mlb_dw'
dsn = f'user={database_username},password={database_password}, host={database_ip},dbname={database_name}'
print(dsn)

user=postgres:postgres,password=postgres, host=172.19.0.2,dbname=mlb_dw


In [21]:

conn = psycopg2.connect(dsn)
with conn:
    with conn.cursor() as curs:
        curs.close()
conn.close

OperationalError: fe_sendauth: no password supplied


In [35]:
with database_connection.connect() as connection:
    # Assuming you have a "users" table with columns "name" and "email"
    table = "box_scores"
    data = json.dumps(players_list[0])
    connection.execute(text(f"INSERT INTO {table} (v_player_game) VALUES ('{data}')"))
connection.close()
